In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
from scipy import *
from scipy.sparse import *
import similaripy as sim
from sklearn.feature_extraction.text import CountVectorizer
import re

In [3]:
import sys
sys.path.append('../')
from xgb_dataset_generation import adding_features
import os

In [4]:
import lightgbm as lgb
import time

Prendiamo un elemento alla volta dal test, facciamo la similarità di quell'elemento ed estraiamo il top50. Quell'elemento poi verrà predetto. Una volta tirata fuori la predizione di quell'elemento, va aggiunto al train [di LightGBM]: non importa rifare la similarità perché valutando un elemento alla volta sappiamo già dalla similarità che facciamo se quell'elemento è nel train o no. 

In [5]:
df_train = pd.read_csv("../dataset/original/train.csv", escapechar="\\")
df_test = pd.read_csv("../dataset/original/test.csv", escapechar="\\")
df_train = df_train.sort_values(by='record_id').reset_index(drop=True)
df_test = df_test.sort_values(by='record_id').reset_index(drop=True)

df_train.linked_id = df_train.linked_id.astype(int)

In [6]:
df_test['linked_id'] = df_test.record_id.str.split("-")
df_test['linked_id'] = df_test.linked_id.apply(lambda x: x[0])
df_test.linked_id = df_test.linked_id.astype(int)
#df_train.linked_id = df_train.linked_id.astype(int)
only_test = set(df_test.linked_id.values) - set(df_train.linked_id.values)
only_test_recordid = df_test[df_test.linked_id.isin(only_test)]
df_test = df_test.drop('linked_id', axis=1)

In [7]:
train1 = pd.read_csv("../dataset/validation_2/train_complete.csv")
train2 = pd.read_csv("../dataset/validation_3/train_complete.csv")
val = pd.read_csv("../dataset/validation/train_complete.csv")

In [8]:
def remove_spaces(s, n=3):
    s = re.sub(' +',' ',s).strip()
    ngrams = zip(*[s[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [8]:
def ngrams_name(test_record, df_train):
    df_train.name = df_train.name.astype(str)
    test_record['name'] = test_record['name'].astype(str)
    corpus = list(df_train.name)
    corpus.append(test_record['name'])
    vectorizer = CountVectorizer(preprocessor = remove_spaces, analyzer=remove_spaces)
    X = vectorizer.fit_transform(corpus)
    X_train = X[:df_train.shape[0],:]
    X_test = X[df_train.shape[0]:,:]
    similarity = sim.jaccard(X_test, X_train.T, k=300)
    return similarity.tocsr()

In [9]:
def ngrams_name_fast(test_record, vectorizer, X_train):
#     deve:
#     - prendere il nome del test
#     - vettorizzarlo
#     - calcolare la similarità
#     - ritornare la similarità con una nuova riga e una nuova colonna
#     - ritornare X_train con una nuova riga (la vettorizzazione del nuovo record)
#     ?
    X_test = vectorizer.transform([test_record['name']])
    similarity = sim.jaccard(X_test, X_train.T, k=300).tocsr()
    return similarity.tocsr(), vstack([X_train,X_test])

In [10]:
def ngrams_address(test_record, df_train):
    df_train.address = df_train.address.fillna('').astype(str)
    test_record.address = test_record.fillna({'address':''}).address
    corpus = list(df_train.address)
    corpus.append(test_record.address)
    vectorizer = CountVectorizer(preprocessor = remove_spaces, analyzer=remove_spaces)
    X = vectorizer.fit_transform(corpus)
    X_train = X[:df_train.shape[0],:]
    X_test = X[df_train.shape[0]:,:]
    similarity = sim.jaccard(X_test, X_train.T, k=300)
    return similarity.tocsr()

In [11]:
def ngrams_address_fast(test_record, vectorizer, X_train):
    test_record.address = test_record.fillna({'address':''}).address
    X_test = vectorizer.transform([test_record.address])
    similarity = sim.jaccard(X_test, X_train.T, k=300)
    return similarity.tocsr(), vstack([X_train,X_test])

In [12]:
def ngrams_email(test_record, df_train):
    df_train.email = df_train.email.fillna('').astype(str)
    test_record.email = test_record.fillna({'email':''}).email
    corpus = list(df_train.email) 
    corpus.append(test_record.email)
    vectorizer = CountVectorizer(preprocessor = remove_spaces, analyzer=remove_spaces)
    X = vectorizer.fit_transform(corpus)
    X_train = X[:df_train.shape[0],:]
    X_test = X[df_train.shape[0]:,:]
    similarity = sim.jaccard(X_test, X_train.T, k=300)
    return similarity.tocsr()

In [24]:
def ngrams_email_fast(test_record, vectorizer, X_train):
    test_record.email = test_record.fillna({'email':''}).email
    X_test = vectorizer.transform([test_record.email])
    similarity = sim.jaccard(X_test, X_train.T, k=300)
    return similarity.tocsr(), vstack([X_train,X_test])

In [14]:
def convert_phones(df_in):
    """
    This functions transforms the phone column from scientific notation to readable string
    format, e.g. 1.2933+E10 to 12933000000
    : param df_in : the original df with the phone in scientific notation
    : return : the clean df
    """
    df = df_in.copy()
    df.phone = df.phone.fillna('').astype(str)
    df.phone = [p.split('.')[0] for p in df.phone]
    return df

def ngrams_phone(test_record, df_train):
    # manually convert test_record phone
    if np.isnan(test_record.phone):
        test_record.phone = test_record.fillna({'phone':''}).phone
    else:
        test_record.phone = test_record.fillna({'phone':''}).phone.astype(str)
    test_record.phone = test_record.phone.split('.')[0]
    df_train = convert_phones(df_train)
    corpus = list(df_train.phone)
    corpus.append(test_record.phone)
    vectorizer = CountVectorizer(preprocessor = remove_spaces, analyzer=remove_spaces)
    X = vectorizer.fit_transform(corpus)
    X_train = X[:df_train.shape[0],:]
    X_test = X[df_train.shape[0]:,:]
    similarity = sim.jaccard(X_test, X_train.T, k=300)
    return similarity.tocsr()

In [15]:
def ngrams_phone_fast(test_record, vectorizer, X_train):
    # manually convert test_record phone
    if np.isnan(test_record.phone):
        test_record.phone = test_record.fillna({'phone':''}).phone
    else:
        test_record.phone = test_record.fillna({'phone':''}).phone.astype(str)
    test_record.phone = test_record.phone.split('.')[0]
    X_test = vectorizer.transform([test_record.phone])
    similarity = sim.jaccard(X_test, X_train.T, k=300)
    return similarity.tocsr(), vstack([X_train,X_test])

In [16]:
def expand_df(df):
    df_list = []
    for (q, pred, pred_rec, score, s_name, s_email, s_phone, s_addr,  idx) in tqdm(
            zip(df.queried_record_id, df.predicted_record_id, df.predicted_record_id_record, df.cosine_score,
                df.name_cosine, df.email_cosine, df.phone_cosine, df.address_cosine, df.linked_id_idx)):
        for x in range(len(pred)):
            df_list.append((q, pred[x], pred_rec[x], score[x], s_name[x], s_email[x], s_phone[x], s_addr[x],  idx[x]))

    # TODO da cambiare predicted_record_id in predicted_linked_id e 'predicted_record_id_record' in 'predicted_record_id'
    df_new = pd.DataFrame(df_list, columns=['queried_record_id', 'predicted_record_id', 'predicted_record_id_record',
                                            'cosine_score', 'name_cosine',
                                            'email_cosine', 'phone_cosine', 'address_cosine', 'linked_id_idx',
                                            ])
    return df_new

In [17]:
def expand_similarities(test_record, vectorizer_name, vectorizer_email, vectorizer_phone, vectorizer_address, 
                        X_train_name, X_train_email, X_train_phone, X_train_address, k=50):
    
    sim_name, X_train_name_new = ngrams_name_fast(test_record, vectorizer_name, X_train_name)
    sim_email, X_train_email_new = ngrams_email_fast(test_record, vectorizer_email, X_train_email)
    sim_phone, X_train_phone_new = ngrams_phone_fast(test_record, vectorizer_phone, X_train_phone)
    sim_address, X_train_address_new = ngrams_address_fast(test_record, vectorizer_address, X_train_address)
        
    hybrid = sim_name + 0.2 * sim_email + 0.2 * sim_phone + 0.2 * sim_address
    
    linid_ = []
    linid_idx = []
    linid_score = []
    linid_name_cosine = []
    linid_email_cosine = []
    linid_phone_cosine = []
    linid_address_cosine = []
    linid_record_id = []
    
    tr = df_train[['record_id', 'linked_id']]
    indices = hybrid.nonzero()[1][hybrid.data.argsort()[::-1]][:k]
    df = tr.loc[indices, :][:k]
    linid_.append(df['linked_id'].values)
    linid_idx.append(df.index)
    linid_record_id.append(df.record_id.values)
    linid_score.append(np.sort(hybrid.data)[::-1][:k]) # Questo ha senso perché tanto gli indices sono sortati in base allo scores di hybrid
    linid_name_cosine.append([sim_name[0, t] for t in indices])
    linid_email_cosine.append([sim_email[0, t] for t in indices])
    linid_phone_cosine.append([sim_phone[0, t] for t in indices])
    linid_address_cosine.append([sim_phone[0, t] for t in indices])
    
    df = pd.DataFrame()
    df['queried_record_id'] = [test_record.record_id]
    df['predicted_record_id'] = linid_
    df['predicted_record_id_record'] = linid_record_id
    df['cosine_score'] = linid_score
    df['name_cosine'] = linid_name_cosine
    df['email_cosine'] = linid_email_cosine
    df['phone_cosine'] = linid_phone_cosine
    df['address_cosine'] = linid_address_cosine
    df['linked_id_idx'] = linid_idx
    
    df_new = expand_df(df)
    
    return df_new, X_train_name_new, X_train_email_new, X_train_phone_new, X_train_address_new

In [18]:
def get_linked_id(new_row):
    new_row['linked_id'] = new_row.record_id.split("-")
    new_row['linked_id'] = new_row.linked_id[0]
    new_row['linked_id'] = int(new_row.linked_id)
    return new_row

# Example

In [36]:
def create_all_vectorizers(df_train):
    # vectorizer Name
    df_train.name = df_train.name.astype(str)
    corpus_name = list(df_train.name)
    vectorizer_name = CountVectorizer(preprocessor = remove_spaces, analyzer=remove_spaces)
    X_train_name = vectorizer_name.fit_transform(corpus_name)

    # vectorizer Email
    df_train.email = df_train.email.fillna('').astype(str)
    corpus_email = list(df_train.email) 
    vectorizer_email = CountVectorizer(preprocessor = remove_spaces, analyzer=remove_spaces)
    X_train_email = vectorizer_email.fit_transform(corpus_email)

    # vectorizer Address
    df_train.address = df_train.address.fillna('').astype(str)
    corpus_address = list(df_train.address)
    vectorizer_address = CountVectorizer(preprocessor = remove_spaces, analyzer=remove_spaces)
    X_train_address = vectorizer_address.fit_transform(corpus_address)

    # vectorizer Phone
    df_train = convert_phones(df_train)
    corpus_phone = list(df_train.phone)
    vectorizer_phone = CountVectorizer(preprocessor = remove_spaces, analyzer=remove_spaces)
    X_train_phone = vectorizer_phone.fit_transform(corpus_phone)
    
    return vectorizer_name, vectorizer_email, vectorizer_address, vectorizer_phone, X_train_name, X_train_email, X_train_address, X_train_phone 

In [37]:
vectorizer_name, vectorizer_email, vectorizer_address, vectorizer_phone, X_train_name, X_train_email, X_train_address, X_train_phone= create_all_vectorizers(df_train)
t1 = time.time()
test_record_exp, X_train_name_new, X_train_email_new, X_train_phone_new, X_train_address_new = expand_similarities(df_test.loc[2], vectorizer_name, vectorizer_email, vectorizer_phone, vectorizer_address, 
                        X_train_name, X_train_email, X_train_phone, X_train_address)
t2 = time.time()
t2-t1

Done: 100%|██████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.04it/s]


1.7972235679626465

In [34]:
test_record_exp

(   queried_record_id  predicted_record_id predicted_record_id_record  \
 0    10000010-TST-CP             10000010                   10000010   
 1    10000010-TST-CP             10099279                10099279-T0   
 2    10000010-TST-CP             10099279                10099279-T4   
 3    10000010-TST-CP             12194552                12194552-T1   
 4    10000010-TST-CP             12194552                12194552-T0   
 5    10000010-TST-CP             12194552                   12194552   
 6    10000010-TST-CP             10192342                   10192342   
 7    10000010-TST-CP             10084458                   10084458   
 8    10000010-TST-CP             10078749                   10078749   
 9    10000010-TST-CP             10078749                10078749-M0   
 10   10000010-TST-CP             10194076                10194076-M2   
 11   10000010-TST-CP             10194076                10194076-M1   
 12   10000010-TST-CP             10194076         

# Create Sequential Test Answers

Dobbiamo creare un test set sequenziale [con le relative risposte], in modo che se ci sono due record che fanno riferimento allo stesso linked_id che però sono presenti solo nel test, il primo che arriva non ha riferimenti nel train ed è impossibile predirlo correttamente; viene però aggiunto al train, dunque quando il secondo record deve essere valutato, allora la risposta corretta corrisponde a quello precedentemente.

In [74]:
group_train = df_train.groupby('linked_id').apply(lambda x: list(x['record_id'])).reset_index().rename(columns={0:'record_ids'})

In [77]:
df_test

,record_id,name,type,address,phone,email,modification,linked_id
0,10000003-TST-MR,"HOTFOCUS CO., LTD.",entity,NaN,4.401574e+11,consulting@outlook.ch,move unique,10000003
1,10000008-TST-M,BONUS TRADE LIMITED,entity,NaN,NaN,help@gmail.gov,move row,10000008
2,10000010-TST-CP,NEW IDEA LIMITED,entity,NaN,1.912469e+10,NaN,identical copy,10000010
3,10000013-TST-MR,VICTORY GROUP LIMITED,entity,NaN,1.949506e+10,NaN,move unique,10000013
4,10000016-TST-MR,"NINGBO RAPID INTERNATIONAL TRADING CO., LTD.",entity,NaN,4.446512e+11,info@outlook.cz,move unique,10000016
5,10000017-TST-MR,ANGELIKA INTERNATIONAL LTD.,entity,NaN,NaN,NaN,move unique,10000017
6,10000018-T4-TST-CP,CHEM D-T Corp.,entity,NaN,3.924568e+09,enquirhs@CHEM.D-T.Corp.ch,identical copy,10000018
7,10000018-TST-M,CHEM D-T Corp.,entity,NaN,3.924568e+09,inquires@CHEM.D-T.Corp.ch,move row,10000018
8,10000020-NV0-TST-CP,JIE LUN INVESTMENT LIMITED,entity,NaN,NaN,NaN,identical copy,10000020
9,10000020-NV1-TST-M,JIE LUN INVESTMENT LIMITED,entity,NaN,3.336016e+11,NaN,move row,10000020


In [78]:
df_test['linked_id'] = df_test.record_id.str.split('-')
df_test['linked_id'] = df_test.linked_id.apply(lambda x: x[0])
df_test.linked_id = df_test.linked_id.astype(int)

In [79]:
df_test = df_test.merge(group_train, how= 'left', on='linked_id')

In [80]:
def seq_labelling(df):
    linked_seen = []
    new_group = { }
    new_df = []
    df_notna = df[~df.record_ids.isna()]
    df_na = df[df.record_ids.isna()]
    relevant_idx= []
    for (i, l, r) in tqdm(zip(df_na.index, df_na.linked_id, df_na.record_id)):
        if l not in linked_seen:
            linked_seen.append(l)
            new_df.append((i, r, np.nan))
            new_group[l] = [r]
        else:
            current_group = new_group[l]
            #print(f'{r} and the group: {current_group}')
            new_df.append((i, r, current_group))
            new_group[l].append(r)
            relevant_idx.append(i)
    res = pd.DataFrame(new_df, columns=['index','record_id', 'record_ids']).set_index('index')
    full_df = pd.concat([df_notna, res])
    full_df = full_df.sort_index()
    return full_df, relevant_idx

def get_target(df):
    df, idx = seq_labelling(df)
    df_notidx = df.loc[~df.index.isin(idx)]
    df = df.loc[idx]
    new_df = []
    for (i,r, g) in tqdm(zip(df.index, df.record_id, df.record_ids)):
        idx = g.index(r)
        g = g[:idx]
        new_df.append((i, r, g))
    new_df = pd.DataFrame(new_df, columns=['index','record_id', 'record_ids']).set_index('index')
    res = pd.concat([new_df, df_notidx])
    res = res.sort_index()
    return res

In [81]:
seq = get_target(df_test[['record_id', 'linked_id','record_ids']])

c:\users\blueace17\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



c:\users\blueace17\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



# Incremental Evaluation

## Nota
Invece di fare le predizioni su tutto il test, visto che sappiamo come performa l'algoritmo classico su quei record che hanno riferimenti nel train, possiamo restringere l'analisi ai record del test che non hanno riferimenti nel train e che hanno duplicati all'interno del test stesso [per quelli che non sono duplicati non ha senso fare tale analisi perché non forniscono nessun contributo alle query successive]

In [97]:
duplicates_df = only_test_recordid[only_test_recordid.duplicated('linked_id', keep=False)]

In [98]:
duplicates_df

,record_id,name,type,address,phone,email,modification,linked_id
338,10000500-T0-TST-M,EAST ASIA PALM hO.c LTD,entity,NaN,4.419548e+11,NaN,move row,10000500
339,10000500-TST-M,"EAST ASIA PALM CO., LTD",entity,NaN,4.419598e+11,NaN,move row,10000500
848,10001224-M0-TST-M,VICTORY SECRET LTD.,entity,NaN,1.900123e+10,support@icloud.gov,move row,10001224
849,10001224-M1-TST-M,VICTORY SECRET LTD.,entity,NaN,NaN,NaN,move row,10001224
850,10001224-TST-M,VICTORY SECRET LTD.,entity,NaN,1.900123e+10,support@icloud.gov,move row,10001224
1592,10002261-NV0-TST-M,"LONGKOU FANLIN NODULAR CAST IRON PIPE CO.,LTD.",entity,6310 WATERBURY RD NANAIMO BC V9V 1M3 CANADA,NaN,NaN,move row,10002261
1593,10002261-TST-M,"LONGKOU FANLIN NODULAR CAST IRON PIPE CO.,LTD.",entity,NaN,4.107031e+12,NaN,move row,10002261
1924,10002752-NV0-TST-M,WALBRAY TRADING LTD.,entity,NaN,NaN,NaN,move row,10002752
1925,10002752-TST-M,WALBRAY TRADING LTD.,entity,NaN,NaN,NaN,move row,10002752
2180,10003124-M0-TST-M,MANDELBAUM LTD. INC.,entity,NaN,NaN,NaN,move row,10003124


In [99]:
idx = duplicates_df.index

In [100]:
def reorder_preds(preds):
    ordered_lin = []
    ordered_score = []
    ordered_record = []
    for i in range(len(preds)):
        l = sorted(preds[i], key=lambda t: t[1], reverse=True)
        lin = [x[0] for x in l]
        s = [x[1] for x in l]
        r = [x[2] for x in l]
        ordered_lin.append(lin)
        ordered_score.append(s)
        ordered_record.append(r)
    return ordered_lin, ordered_score, ordered_record

In [115]:
X_train_name

<691450x100241 sparse matrix of type '<class 'numpy.int64'>'
	with 14180025 stored elements in Compressed Sparse Row format>

In [118]:
vgfsd = vectorizer_name.transform(['vgfsd'])
vstack([X_train_name,vgfsd])

<691451x100241 sparse matrix of type '<class 'numpy.int64'>'
	with 14180028 stored elements in Compressed Sparse Row format>

In [120]:
# Re-load training set
df_train = pd.read_csv("../dataset/original/train.csv", escapechar="\\")
df_train = df_train.sort_values(by='record_id').reset_index(drop=True)
df_train.linked_id = df_train.linked_id.astype(int)
# get vectorizers
vectorizer_name, vectorizer_email, vectorizer_address, vectorizer_phone, X_train_name, X_train_email, X_train_address, X_train_phone= create_all_vectorizers(df_train)

# Online Learning    -------------> TODO: ci mette troppo, circa 7ore
test_prediction = pd.DataFrame()
seen = []

for i in tqdm(idx[:10]):
    test_row = duplicates_df.loc[i]
    print(f'Record: {test_row.record_id}')
    t1 = time.time()
    #print(f'Extract test record at index: {i}')
    # Get the test record to be evaluate
    #print('Create Features')
    test_row_exp, X_train_name, X_train_email, X_train_phone, X_train_address = expand_similarities(test_row, vectorizer_name, vectorizer_email, vectorizer_phone, vectorizer_address, 
                        X_train_name, X_train_email, X_train_phone, X_train_address)
    test_row_exp = adding_features(test_row_exp, isValidation=False, path=os.path.join('..', 'dataset', 'original'), incremental_train=df_train)
    
    # Get predictions
    #print('Get Predictions')
    predictions = ranker.predict(test_row_exp.drop(['queried_record_id','linked_id_idx', 'predicted_record_id','predicted_record_id_record'], axis=1))
    test_row_exp['predictions'] = predictions
    df_predictions = test_row_exp[['queried_record_id', 'predicted_record_id', 'predicted_record_id_record', 'predictions']]
    
    # Re-order predictions
    #print('Reorder Predictions')
    rec_pred = []
    for (l,p,record_id) in zip(df_predictions.predicted_record_id, df_predictions.predictions, df_predictions.predicted_record_id_record):
        rec_pred.append((l, p, record_id))

    df_predictions['rec_pred'] = rec_pred
    group_queried = df_predictions[['queried_record_id', 'rec_pred']].groupby('queried_record_id').apply(lambda x: list(x['rec_pred']))
    df_predictions = pd.DataFrame(group_queried).reset_index().rename(columns={0 : 'rec_pred'})
    
    # Store predictions
    #print('Store Predictions')
    df_predictions['ordered_linked'], df_predictions['ordered_scores'], df_predictions['ordered_record'] = reorder_preds(df_predictions.rec_pred.values)
    test_prediction = pd.concat([test_prediction, df_predictions], ignore_index=True)
    new_row = get_linked_id(test_row)
    df_train = df_train.append(new_row, ignore_index=True)
    t2 = time.time()
    #print(f'Iteration completed in {t2 - t1}s')

Record: 10000500-T0-TST-M








  0%|                                                                                            | 0/1 [00:00<?, ?it/s]





Preprocessing:   0%|                                                                             | 0/1 [00:00<?, ?it/s]





Allocate memory per threads:   0%|                                                               | 0/1 [00:00<?, ?it/s]





Allocate memory per threads: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.07it/s]





Build coo matrix: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.95it/s]





Done: 100%|██████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.93it/s]





Done: 100%|██████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.92it/s]





  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

NaN on queried_name: 0
Nan on predicted_name: 0








  0%|                                                                                           | 0/50 [00:00<?, ?it/s]





100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 50135.12it/s]

c:\users\blueace17\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\blueace17\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\blueace17\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

...







  0%|                                                                                           | 0/50 [00:00<?, ?it/s]





100%|██████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<?, ?it/s]

Record: 10003124-M0-TST-M








  0%|                                                                                            | 0/1 [00:00<?, ?it/s]





Preprocessing:   0%|                                                                             | 0/1 [00:00<?, ?it/s]





Allocate memory per threads:   0%|                                                               | 0/1 [00:00<?, ?it/s]





Allocate memory per threads: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.02it/s]





Build coo matrix: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.91it/s]





Done: 100%|██████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.89it/s]





Done: 100%|██████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.89it/s]





  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

NaN on queried_name: 0
Nan on predicted_name: 0








  0%|                                                                                           | 0/50 [00:00<?, ?it/s]





100%|██████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<?, ?it/s]

In [121]:
test_prediction

,queried_record_id,rec_pred,ordered_linked,ordered_scores,ordered_record
0,10000500-T0-TST-M,"[(10112571, -8.151580851741322, 10112571-NV1),...","[10175527, 10206269, 11011306, 11011306, 11011...","[-7.799285036108848, -7.955492562070846, -8.05...","[10175527, 10206269, 11011306, 11011306-NV0, 1..."
1,10000500-TST-M,"[(10000500, 3.710556417215613, 10000500-T0-TST...","[10000500, 10206269, 10123305, 10123305, 10004...","[3.710556417215613, -7.9057710975532665, -8.11...","[10000500-T0-TST-M, 10206269, 10123305-M0, 101..."
2,10001224-M0-TST-M,"[(10000126, -7.174336442397593, 10000126), (12...","[10024018, 10024018, 10024018, 12156157, 10000...","[-5.923994129651748, -5.923994129651748, -5.92...","[10024018-M2, 10024018-M1, 10024018-M0, 121561..."
3,10001224-M1-TST-M,"[(10001224, 3.5712758853865507, 10001224-M0-TS...","[10001224, 10024018, 10024018, 10024018, 12090...","[3.5712758853865507, -4.670545844145791, -4.67...","[10001224-M0-TST-M, 10024018-M0, 10024018-M1, ..."
4,10001224-TST-M,"[(10001224, 6.782490107512067, 10001224-M0-TST...","[10001224, 10001224, 10024018, 10024018, 10024...","[6.782490107512067, 3.3053680193312114, -5.923...","[10001224-M0-TST-M, 10001224-M1-TST-M, 1002401..."
5,10002261-NV0-TST-M,"[(10030441, -8.314921415728369, 10030441-T1), ...","[10107248, 10007988, 10004245, 10025548, 10025...","[-7.87958792238874, -7.98112896869165, -7.9971...","[10107248, 10007988, 10004245, 10025548-M1, 10..."
6,10002261-TST-M,"[(10002261, 4.0371570058777095, 10002261-NV0-T...","[10002261, 10107248, 10025548, 10004245, 10025...","[4.0371570058777095, -8.16763342152442, -8.177...","[10002261-NV0-TST-M, 10107248, 10025548-M0, 10..."
7,10002752-NV0-TST-M,"[(10000670, 0.846763644157856, 10000670), (100...","[10000670, 10000670, 10040746, 10040746, 10040...","[0.846763644157856, 0.846763644157856, 0.47950...","[10000670, 10000670-NV0, 10040746-T3, 10040746..."
8,10002752-TST-M,"[(10002752, 1.1569710723194153, 10002752-NV0-T...","[10002752, 10000670, 10000670, 10040746, 10040...","[1.1569710723194153, 0.846763644157856, 0.8467...","[10002752-NV0-TST-M, 10000670-NV0, 10000670, 1..."
9,10003124-M0-TST-M,"[(10005938, -8.311967598500814, 10005938), (10...","[10172971, 10172971, 10172971, 10025917, 10025...","[-7.765717178343136, -7.765717178343136, -7.76...","[10172971-M0, 10172971-M2, 10172971-M1, 100259..."


L'idea era che una volta calcolate le predizioni 'sequenziali', possiamo guardare quanto queste siano corrette matchandole con seq['record_ids']

# Problemi:
- ci mette davvero troppo, soprattutto il calcolo delle similarità ed estrazione delle topN, trovare un modo più rapido